## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

**I. Промежуточная задача**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

def get_dataset(path_to_data, count = 0):
  likes_data_list = []

  with open(path_to_data, 'r') as f:
    lines = f.readlines()

    for idx, line in enumerate(lines):
      if idx > count:
        break
      items = [int(i) for i in line.split()]
      likes_data = np.empty((len(items), 3), dtype=int)
      likes_data[:, 0] = idx # user_id
      likes_data[:, 1] = items[::-1] # track_id
      likes_data[:, 2] = np.arange(len(items)) # order
      likes_data_list.append(likes_data)
    
  array_data = np.vstack(likes_data_list)

  df = pd.DataFrame(array_data, columns = ['user_id', 'track_id', 'order'])

  return df

In [ ]:
path_to_data = '/content/drive/MyDrive/hw2/test'

example = get_dataset(path_to_data, count = 1000)
example.head() 

,user_id,track_id,order
0,0,471705,0
1,0,219426,1
2,0,101168,2
3,0,361110,3
4,0,22932,4


**II. Итоговая задача**

In [ ]:
from sklearn.model_selection import KFold

class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed = 23):
      self.n_folds = n_folds
      self.p = p
      self.random_seed = random_seed
    
    def split(self, df):
      df = df.copy()  
      users = df['user_id'].unique()
      # разбивка по фолдам
      users_kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_seed)

      for train_users, test_users in users_kfold.split(users):
        train_mask = df['user_id'].isin(train_users)
        test_mask = df['user_id'].isin(test_users) & (df['order'] < self.p)
        yield train_mask, test_mask

In [ ]:
n_folds = 3
p = 1
cv = UsersKFoldPOut(n_folds = n_folds, p = p)

for i, (train_mask, test_mask) in enumerate(cv.split(example)):
  train = example[train_mask]
  test = example[test_mask]

  # Проверка на отсутствие общих пользователей между train и test
  assert set() == set(train['user_id']).intersection(test['user_id'])
  # Проверка на соответствие условию: "В test должно быть не более p последних треков (параметр класса p)"
  assert test[test['order'] > p].shape[0] == 0

  print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 56853, Test: 334
Fold#1 | Train: 56087, Test: 334
Fold#2 | Train: 57452, Test: 333
